In [230]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/refs/heads/master/data/tinyshakespeare/input.txt

--2026-02-21 12:11:00--  https://raw.githubusercontent.com/karpathy/char-rnn/refs/heads/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.3’

input.txt.3         100%[===================>]   1.06M  5.08MB/s    in 0.2s    

2026-02-21 12:11:01 (5.08 MB/s) - ‘input.txt.3’ saved [1115394/1115394]



In [231]:
import torch
import torch.nn as nn
from torch.nn import functional as F
# torch.manual_seed(1337)
device = 'cuda' if torch.cuda.is_available() else 'mps' # macOS

n_embd = 32 # 384 = batch_size * block_size (64 * 256)
n_layer = 4 # 6
dropout = 0.2
batch_size = 4 # 64, how many independent sequences will be process in parallel?
block_size = 8 # 256, what is the maximum context length for prediction?
n_head = 4 # 6
learning_rate = 1e-3 # 3e-4
max_iter = 1000 # 5000
# eval_interval = 500
# eval_iters = 200

In [232]:
# Data preparation

with open('input.txt', 'r', encoding='utf-8') as fp:
    text = fp.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
# split the data into training and validation set
n = int(len(data) * 0.9)
train_data = data[:n]
val_data = data[n:]

In [233]:
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [234]:
class Head(nn.Module):
    """ One head of self attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)

        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x) # (B, T, C)
        q = self.query(x) # (B, T, C)
        v = self.value(x) # (B, T, C)

        # compute attention scores
        wei = q @ k.transpose(-2, -1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)

        # perform the weighted aggregation of the values
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

In [235]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel"""

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [236]:
class FeedForward(nn.Module):
    """ a simple linear layer followed by a non-linarity"""

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout)
        )
    
    def forward(self, x):
        return self.net(x)

In [ ]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation"""

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_had: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
    
    def forward(self, x):
        x = x + self.sa(self.ln1(x)) # residual value
        x = x + self.ffwd(self.ln2(x)) # residual value
        return x

In [ ]:
class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        #  each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd) # (T, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd) # (T, n_embd)
        # self.sa_heads = MultiHeadAttention(4, n_embd//4) # i.e., 4 heads of 8-dimensional self-attention
        # self.ffwd = FeedForward(n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.lm_head = nn.Linear(n_embd, vocab_size) # language model head
    
    def forward(self, idx, targets=None):
        B, T = idx.shape
        # idx and targets are both (B, T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B, T, C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) #(T, C)
        x = tok_emb + pos_emb # (B, T, C)
        x = self.blocks(x) # (B, T, C)
        logits = self.lm_head(x) # (B, T, vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            # Targets shape: (B, T)
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, _ = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probablities
            probs = F.softmax(logits, dim=1) # (B, C)
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [239]:
model = BigramLanguageModel()
mm = model.to(device)
optimizer = torch.optim.AdamW(mm.parameters(), lr=learning_rate)

for steps in range(max_iter):
    # sample a batch of data
    xb, yb = get_batch('train')

    # evluate the loss
    logits, loss = mm(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
    if steps % 500 == 0 or steps == max_iter - 1:
        print(f'{steps}: loss: {loss.item():.4f}')

print(decode(mm.generate(torch.zeros((1, 1), dtype=torch.long, device=device), max_new_tokens=500)[0].tolist()))
print(sum(p.numel() for p in mm.parameters())/1e6, "M parameters")

0: loss: 4.5023
500: loss: 3.2678
999: loss: 2.7806


Sush, v Who s emerie haro gh to r thend F man Crde lounc aicn an sll.
Anguth scepitdnd. d otek tshit twor ait t an
Tho gh Ce be bano he;
Akar thugo.
L s LyI oaleee totd sok o mopto
BA y R thilow hting!p me I,
Anot.


Ws soreele ke the t too

And b'dou


Whe me'f s ce boous be,

Whee.
H

The om fre bedI ffyo fffou a y d t tithy hepe;



ouy avic w y :


E IUAay.
jo
No

LWelie an
Aaf se lio lof s;ok t peit hind o arpk sheeov t peee;


Whi W ve herit, t I! o;o wo taths y,

:
F Why ouBath
Dch hs,iv
0.054913 M parameters
